파일 경로는 수정해야합니다.
두 클래스를 엮어서 나타낸 것.

In [3]:
import os

json_dir = "./train/reduced/" #현재 코드가 있는 디렉토리로부터 json 파일 있는 경로
txt_dir ="./txt/"
jsonFiles = os.listdir(json_dir) #json_dir 경로에 있는 모든 파일, 디렉토리 이름을 리스트로 변환(순서는 안정해져 있음)
print(jsonFiles)

['01_sou_001106_005526_front_03-2onepiece(jumpsuite)_woman.json', '01_sou_001106_005527_back_03-2onepiece(jumpsuite)_woman.json', '01_sou_001106_005528_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_001106_005529_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_001106_005530_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_003051_015251_front_03-2onepiece(jumpsuite)_man.json', '01_sou_003051_015252_back_03-2onepiece(jumpsuite)_man.json', '01_sou_003051_015253_wear_03-2onepiece(jumpsuite)_man.json', '01_sou_006055_030271_front_03-2onepiece(jumpsuite)_woman.json', '01_sou_006055_030272_back_03-2onepiece(jumpsuite)_woman.json', '01_sou_006055_030273_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_006057_030281_front_03-2onepiece(jumpsuite)_woman.json', '01_sou_006057_030282_back_03-2onepiece(jumpsuite)_woman.json', '01_sou_006057_030283_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_006057_030284_wear_03-2onepiece(jumpsuite)_woman.json', '01_sou_008687_043431_front_03-2onepiece(

In [ ]:
washing_method_class = {
    "Hand Washing30" : 0,
    "Washing30" : 1,
    "Washing40" : 2, # "Washing40_1"
    "Washing60" : 3,
    "Washing95" : 4,


    # do not washing or washing method == ‘none’
    "DRYCLEAN" : 5,
    "DRYCLEAN : LAUNDRY" : 6,
    "DRYCLEAN : petroleum solvent only" : 7
}
fiber_class = {
    "Cotton" : 0,
    "Hemp" : 1,
    "cellulose fiber Others" : 2,
    "Silk" : 3,
    "Wool" : 4,
    "protein fiber Others" : 5,
    "Viscos rayon" : 6,
    "regenerated fiber Others" : 7,
    "Polyester" : 8,
    "Nylon" : 9,
    "Polyurethane" :10,
    "synthetic fiber Others" : 11
}


In [4]:
import json

for idx in range(0,len(jsonFiles)): #원단별 세탁방법, 데이터 수, 드라이클리닝 개수 통계 내기
    with open(json_dir + jsonFiles[idx], encoding='utf-8') as file: #파일 읽기(json파일 있는 경로 + 파일 이름)
        data=json.load(file)
    fiber = data["fiber_composition"].split(",") #어노테이션 가져오기
    washing_method = data["washing_method"].strip() 
    drycleaning = "drycleaning".strip()
    if washing_method == "do not washing" or washing_method=='none':
        washing_method=drycleaning

    #폴리곤 데이터를 통해서 bbox 얻기
    annotations = data['annotation']
    image_width = data['dataset']['dataset.width']
    image_height = data['dataset']['dataset.height']
    yolo_data = []
    for annotation in annotations:
        points = annotation['annotation_point']
        xs = points[0::2]  # Extract x coordinates
        ys = points[1::2]  # Extract y coordinates

        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)

    bbox_width = max_x - min_x
    bbox_height = max_y - min_y

    x_center = (min_x + bbox_width / 2.0) / image_width
    y_center = (min_y + bbox_height / 2.0) / image_height
    norm_width = bbox_width / image_width
    norm_height = bbox_height / image_height

    #class id 설정
    subClass_id=[] 
    if isinstance(fiber, str): #원단이 하나인지 확인
        class_id = 8*fiber_class[fiber] + washing_method_class[washing_method]
    else: #원단이 여러개면 subCalss_id에 각 class_id append.
        for many_fiber in fiber:
            class_id = 8*fiber_class[many_fiber] + washing_method_class[washing_method]
            subClass_id.append(class_id)

    f = open(txt_dir + "{}.txt".format(idx), "w+") #w+로 인해 있으면 대체, 없으면 새로 만듦.
    for id in subClass_id:
        f.write(str(id) + " " + str(x_center) + " " + str(y_center) + " " + str(norm_width) + " " + str(norm_height))
    
    f.write("\n")
    f.close()
    